Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [50]:
from random import random, seed
from itertools import product
import numpy as np

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [51]:
UNIVERSE_SIZE = 100_000
NUM_SETS = 10_000
DENSITY = 0.3

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [52]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [53]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Greedy algorithm

In [54]:
def greedy_set_cover(SETS, COSTS, UNIVERSE_SIZE):
    covered = np.zeros(UNIVERSE_SIZE, dtype=bool) 
    solution = []

    while not np.all(covered): 
        best_set = None
        best_ratio = -1
        
        for i, current_set in enumerate(SETS):
            if i in solution:
                continue  
            
            uncovered_elements = np.logical_and(current_set, ~covered)  # Uncovered elements in this set
            num_uncovered = np.sum(uncovered_elements)  # Count uncovered elements
            
            if num_uncovered > 0:
                ratio = num_uncovered / COSTS[i]
                if ratio > best_ratio:
                    best_ratio = ratio
                    best_set =i
        
        solution.append(best_set)
        covered = np.logical_or(covered, SETS[best_set]) 
    return solution

In [55]:
solution = greedy_set_cover(SETS,COSTS,UNIVERSE_SIZE)
valid(solution), cost(solution)

(np.True_, np.float64(1761947.4472252252))

## Hill Climber

In [56]:
def tweak(solution):
    new_solution = solution.copy()
    index = rng.integers(0, NUM_SETS)
    new_solution[index] = not new_solution[index]
    return new_solution

In [57]:
best_solution = np.full(NUM_SETS, True)
num_iters = 2

for step in range(num_iters):
    solution = np.full(NUM_SETS, False)
    
    while not valid(solution):
        solution = tweak(solution)
    
    if cost(solution) < cost(best_solution):
        best_solution = solution.copy()
        
print(valid(best_solution), cost(best_solution))

True 2775569.4005117496
